In [ ]:
import random
import numpy as np
import pickle
import json

!pip install flask_ngrok

from flask import Flask, render_template, request
from flask_ngrok import run_with_ngrok
import nltk
from keras.models import load_model
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()


In [ ]:
#connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install tensorflow


In [ ]:
from tensorflow.keras.models import load_model
model = load_model('/content/drive/MyDrive/ColabModels/chatbot_v3.h5')

In [ ]:
from tensorflow.keras.models import load_model
model = load_model('chatbot_v3.h5')

In [ ]:
#intialising chat & loading up the chatbot model trained earlier
model = load_model("chatbot_v3.h5")

data_file = open("intents.json").read()
words = pickle.load(open("words.pkl", "rb"))
classes = pickle.load(open("classes.pkl", "rb"))

app = Flask(__name__)

In [ ]:
#creates a route for an HTML interface and to handle responses

@app.route("/")
def home():
  return render_template("index.html")

@app.route("/get", methods=["POST"])
def chatbot_response():
  msg = request.form["msg"]

  data_file = open("intents.json").read()
  intents = json.loads(data_file)

  if msg.startswith('my name is'):
    name = msg[11:]
    ints = predict_class(msg, model)
    resl = getResponse(ints, intents)
    res = resl.replace("{n}", name)
  elif msg.startswith('hi my name is'):
    name = msg[14:]
    ints = predict_class(msg, model)
    resl = getResponse(ints, intents)
    res = resl.replace("{n}", name)
  else:
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
  return res

In [ ]:
#chat functionalities

def clean_up_sentence(sentence):
  sentence_words = nltk.word_tokenize(sentence)
  sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
  return sentence_words


#return a BoW -> 0 or 1 if a word in the bag exists in the sentence

def bow(sentence, words, show_details=True):
  sentence_words = clean_up_sentence(sentence)

  bag = [0] * len(words)
  for s in sentence_words:
    for i, w in enumerate(words):
      if w == s:
        bag[i] = l
        if show_details:
          print("found in bagL %s" % w)
  return np.array(bag)



In [ ]:
#analyse input text and filter predictions based on a threshold.
#sort and return intents w/ probabilities exceeding the threshold

def predict_class(sentence, model):
  p = bow(sentence, words, show_details=False)
  res = model.predict(np.array([p]))[0]
  ERROR_THRESHOLD = 0.25
  results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

  results.sort(key=lambda x: x[l], reverse=True)
  return_list = []
  for r in results:
    return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
  return return_list


#retrieve responses based on highest probability
#select from predefined sources in the intents_json

def getResponse(ints, intents_json):
  tag = ints[0]["intent"]
  list_of_intents = intents_json["intents"]
  for i in list_of_intents:
    if i["tag"] == tag:
      result = random.choice(i["responses"])
      break
    return result


In [ ]:
!pip install waitress

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 kB 2.3 MB/s eta 0:00:00


In [ ]:
#run the chatbot app/UI

if __name__ == "__main__":
  app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
